# Midi devices

## PyGame

    pygame.midi.get_device_info(an_id)
    Docstring:
    returns information about a midi device
    pygame.midi.get_device_info(an_id): return (interf, name,
                                                input, output,
                                                opened)

    interf - a text string describing the device interface, eg 'ALSA'.
    name - a text string for the name of the device, eg 'Midi Through Port-0'
    input - 0, or 1 if the device is an input device.
    output - 0, or 1 if the device is an output device.
    opened - 0, or 1 if the device is opened.

    If the id is out of range, the function returns None.

In [1]:
import sys, pygame, pygame.midi
import json
 
# set up pygame
pygame.init()
pygame.midi.init()

midi_devices = {
    "input": [],
    "output": []
}

# list all midi devices
for x in range(pygame.midi.get_count()):
    device = pygame.midi.get_device_info(x)
    
    # decode to utf 8
    device = list(device)
    device[0] = device[0].decode("utf-8") # interface 
    device[1] = device[1].decode("utf-8") # device name
    device = tuple(device)
    
    if device[2]:
        midi_devices["input"].append(device)
        
    if device[3]:
        midi_devices["output"].append(device)
    

midi_devices

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


{'input': [('MMSystem', 'SAMSUNG_Android', 1, 0, 0)],
 'output': [('MMSystem', 'Microsoft MIDI Mapper', 0, 1, 0),
  ('MMSystem', 'Microsoft GS Wavetable Synth', 0, 1, 0),
  ('MMSystem', 'SAMSUNG_Android', 0, 1, 0)]}

In [2]:
midi_devices["input"][0][0]

'MMSystem'

If no midi devices `get_count() = -1`

In [30]:
pygame.midi.get_count()

4

In [31]:
pygame.midi.quit()

In [ ]:
import sys, pygame, pygame.midi
 
# set up pygame
pygame.init()
pygame.midi.init()

# list all midi devices
for x in range( 0, pygame.midi.get_count() ):
    print(pygame.midi.get_device_info(x))

# open a specific midi device
inp = pygame.midi.Input(1)

# run the event loop
while True:
    if inp.poll():
     # no way to find number of messages in queue
     # so we just specify a high max value
        print(inp.read(1000))

 # wait 10ms - this is arbitrary, but wait(0) still resulted
 # in 100% cpu utilization
pygame.time.wait(10)

(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0)
(b'MMSystem', b'SAMSUNG_Android', 1, 0, 0)
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0)
(b'MMSystem', b'SAMSUNG_Android', 0, 1, 0)
[[[144, 48, 64, 0], 27705]]
[[[128, 48, 64, 0], 29304]]


# Mido
https://mido.readthedocs.io/en/latest/lib.html#mido.get_input_names

In [4]:
import mido
mido.get_input_names()

['SAMSUNG_Android 0']

In [5]:
mido.get_output_names()

['Microsoft GS Wavetable Synth 0', 'SAMSUNG_Android 1']

In [6]:
mido.get_ioport_names()

[]

In [9]:
midi_devices = {
    "input": [],
    "output": []
}

midi_devices["input"] = mido.get_input_names()
midi_devices["output"] = mido.get_output_names()

midi_devices

{'input': ['SAMSUNG_Android 0'],
 'output': ['Microsoft GS Wavetable Synth 0', 'SAMSUNG_Android 1']}

open messages for 3 seconds

In [4]:
from datetime import datetime, timedelta
import time

In [5]:
start = datetime.now()
finish = start + timedelta(seconds=3)

inport = mido.open_input('SAMSUNG_Android 0')

for msg in inport:
    print(msg)
    
    if datetime.now() > finish:
        break
        
print("Finished")

note_on channel=0 note=48 velocity=64 time=0
note_off channel=0 note=48 velocity=64 time=0
Finished


In [9]:
inport.close()

# rtmidi
https://spotlightkid.github.io/python-rtmidi/rtmidi.html

In [8]:
%pip install python-rtmidi

In [1]:
import rtmidi

In [2]:
midiin = rtmidi.MidiIn()
midiin.get_port_count()

1

In [3]:
midiin.get_port_name(0)

'MIDI function 0'

In [66]:
midiin = rtmidi.MidiIn()
for n in range(midiin.get_port_count()):
    print(n, midiin.get_port_name(n))
    
print(n, midiin.get_ports())

0 MIDI function 0
0 ['MIDI function 0']


In [68]:
midiin.get_current_api()

4

In [14]:
midiout = rtmidi.MidiOut()
midiout.get_port_count()

2

In [15]:
for n in range(midiout.get_port_count()):
    print(n, midiout.get_port_name(n))
    
print(n, midiout.get_ports())

0 Microsoft GS Wavetable Synth 0
1 SAMSUNG_Android 1
1 ['Microsoft GS Wavetable Synth 0', 'SAMSUNG_Android 1']


In [92]:
import json
midiin = rtmidi.MidiIn()
midiout = rtmidi.MidiOut()

midi_devices = {
    "input": [midiin.get_port_name(n) for n in range(midiin.get_port_count())],
    "output": [midiout.get_port_name(n) for n in range(midiout.get_port_count())]
}

print(json.dumps(midi_devices, indent=2))

{
  "input": [
    "MIDI function 0"
  ],
  "output": [
    "Microsoft GS Wavetable Synth 0",
    "MIDI function 1"
  ]
}


In [5]:
rtmidi.__version__

'1.4.7'

In [73]:
rtmidi.get_compiled_api()

[4]

In [84]:
rtmidi.get_compiled_api_by_name("winmm")

4

    API_UNSPECIFIED  0
    API_MACOSX_CORE  1
    API_LINUX_ALSA   2
    API_UNIX_JACK    3
    API_WINDOWS_MM   4    
    API_RTMIDI_DUMMY 5

In [60]:
rtmidi.API_WINDOWS_MM

4

In [110]:
for n in range(6):
    name = rtmidi.get_api_name(n)
    name_complete = rtmidi.get_api_display_name(n)
    print(f"{name} ({name_complete})")

unspecified (Unknown)
core (CoreMidi)
alsa (ALSA)
jack (Jack)
winmm (Windows MultiMedia)
dummy (Dummy)


In [82]:
rtmidi.get_api_display_name(rtmidi.API_WINDOWS_MM)

'Windows MultiMedia'

In [62]:
from rtmidi import midiutil

In [112]:
midiutil.get_api_from_environment()

0

In [113]:
midiutil.list_input_ports()

Available MIDI input ports:

[0] MIDI function 0



In [91]:
midiutil.list_output_ports(api=5)

Available MIDI ouput ports:

[0] Microsoft GS Wavetable Synth 0
[1] MIDI function 1



# CSound
http://www.csounds.com/manual/html/CommandFlags.html

In [40]:
!csound --midi-devices

0dBFS level = 32768.0


--Csound version 6.15 (double samples) Aug 13 2020
[commit: 18c2c7897425f462b9a7743cee157cb410c88198]
libsndfile-1.0.29pre1
rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
1 MIDI input devices
 0: 0 (SAMSUNG_Android)
3 MIDI output devices
 0: 0 (Microsoft MIDI Mapper)
 1: 1 (Microsoft GS Wavetable Synth)
 2: 2 (SAMSUNG_Android)
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.092s, CPU: 0.092s


In [42]:
import sys
import ctcsound

cs = ctcsound.Csound()
result = cs.compile_("--midi-devices")


In [43]:
result

-1

# ctcsound

https://csound.com/docs/ctcsound/ctcsound-API.html#ctcsound.Csound.midiDevList

http://www.csounds.com/manual/html/CommandFlags.html

In [1]:
import ctcsound

In [2]:
cs = ctcsound.Csound()

In [7]:
cs.start()

0

In [32]:
cs.audioDevList(True)

[{'device_name': 'Asignador de sonido Microsoft - Output [MME]',
  'device_id': 'dac0',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True},
 {'device_name': 'Altavoz/Auricular (Realtek High [MME]',
  'device_id': 'dac1',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True},
 {'device_name': 'Controlador primario de sonido [Windows DirectSound]',
  'device_id': 'dac2',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True},
 {'device_name': 'Altavoz/Auricular (Realtek High Definition Audio) [Windows Dir',
  'device_id': 'dac3',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True},
 {'device_name': 'Altavoz/Auricular (Realtek High Definition Audio) [Windows WAS',
  'device_id': 'dac4',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True},
 {'device_name': 'Speakers (Realtek HD Audio output) [Windows WDM-KS]',
  'device_id': 'dac5',
  'rt_module': 'PortAudio',
  'max_nchnls': 2,
  'isOutput': True}]

In [16]:
cs.midiDevList(True)

NameError: name 'csoundMidiDevice' is not defined

In [19]:
cs.stop()


# Pyrtmidi
Looks interesting, but needs further installation for C++

https://github.com/patrickkidd/pyrtmidi

In [40]:
import rtmidi

midiin = rtmidi.RtMidiIn()

def print_message(midi):
    if midi.isNoteOn():
        print('ON: ', midi.getMidiNoteName(midi.getNoteNumber()), midi.getVelocity())
    elif midi.isNoteOff():
        print('OFF:', midi.getMidiNoteName(midi.getNoteNumber()))
    elif midi.isController():
        print('CONTROLLER', midi.getControllerNumber(), midi.getControllerValue())

ports = range(midiin.getPortCount())
if ports:
    for i in ports:
        print(midiin.getPortName(i))
    print("Opening port 0!") 
    midiin.openPort(0)
    while True:
        m = midiin.getMessage(250) # some timeout in ms
        if m:
            print_message(m)
else:
    print('NO MIDI INPUT PORTS!')

AttributeError: module 'rtmidi' has no attribute 'RtMidiIn'

# py-midi

In [ ]:
%pip uninstall py-midi

In [29]:
import midi

In [33]:
midi.midi.MidiConnector()

1

In [101]:
idx = [4, 5, 0]
names = ["win", "linux", "mac"]

In [104]:
api = []
for idx, name in zip(idx, names):
    dev = {"idx": idx,
           "name": name}
    api.append(dev.copy())

In [105]:
api

[{'idx': 4, 'name': 'win'},
 {'idx': 5, 'name': 'linux'},
 {'idx': 0, 'name': 'mac'}]